In [1]:
from fmm import FastMapMatch,Network,NetworkGraph,UBODTGenAlgorithm,UBODT,FastMapMatchConfig
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig
from shapely.geometry import LineString,shape, mapping

In [2]:
# Read network and graph
network = Network("network-Porto/edges.shp","fid", "u", "v")
print "Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count())
graph = NetworkGraph(network)

Nodes 908 edges 1675


In [3]:
# Precompute UBODT
# Can be skipped if you already generated an ubodt file
ubodt_gen = UBODTGenAlgorithm(network,graph)
# The delta is defined as 3 km approximately. 0.03 degrees. 
status = ubodt_gen.generate_ubodt("network-Porto/ubodt.txt", 0.03, binary=False, use_omp=True)
# Binary is faster for both IO and precomputation
# status = ubodt_gen.generate_ubodt("stockholm/ubodt.bin", 0.03, binary=True, use_omp=True)
print status

Status: success
Time takes 0.519 seconds



In [3]:
# Create FMM model
### Read UBODT
ubodt = UBODT.read_ubodt_csv("network-Porto/ubodt.txt")
### Create FMM model
fmm_model = FastMapMatch(network,graph,ubodt)

k = 8
radius = 0.003
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k,radius,gps_error)

In [4]:
import os, glob
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm

In [35]:
a = np.zeros((8760, 3350))
b = [[] for row in range(1675)]
start = datetime.datetime(2013, 7, 1, 0, 0, 0)
# FMM
for t in tqdm(range(13)):
    df = pd.read_csv('data/tra' + str(t) + '.csv')
    for wkt, ti in zip(df['geometry'], df['TIMESTAMP']):

        result = fmm_model.match_wkt(wkt, fmm_config)

        candidates = []
        for c in result.candidates:
            candidates.append((c.edge_id,c.source,c.target,c.error,c.length,c.offset,c.spdist,c.ep,c.tp))

        df_c = pd.DataFrame(candidates, columns=["eid","source","target","error","length","offset","spdist","ep","tp"])
        df_c['speed'] = df_c['spdist'] * 24000
        
        now_time = ti
        if now_time.count(':') == 1:
            now_time += ':00'
        cnow_time = datetime.datetime.strptime(now_time, "%Y-%m-%d %H:%M:%S")
        
        for i in range(len(df_c) - 1):
            if i != 0 or i != len(df_c) - 1:
                if df_c['eid'][i] != df_c['eid'][i+1]:
                    if df_c['speed'][i] >= 5 and df_c['speed'][i] < 120:
                        b[int(df_c['eid'][i])].append(str(round(df_c['speed'][i], 2)))
                        index = int((((cnow_time - start).total_seconds()) / 60) // 60)
                        a[index][int(df_c['eid'][i])*2] += 1
                        a[index][int(df_c['eid'][i])*2 + 1] += df_c['speed'][i]
            cnow_time += datetime.timedelta(seconds=15)

100%|██████████| 13/13 [1:50:39<00:00, 510.74s/it]


In [36]:
tf = pd.DataFrame(a)
tf.to_csv('data/traffic_info.csv', index=False)

,0,1,2,3,4,5,6,7,8,9,...,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349
0,2.0,39.429934,0.0,0.000000,4.0,133.672792,1.0,34.812037,1.0,34.725719,...,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,26.834133,0.0,0.0
1,2.0,50.572508,1.0,31.917500,2.0,61.683941,1.0,12.877057,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,2.0,34.161755,0.0,0.0
2,3.0,67.625311,0.0,0.000000,2.0,36.999047,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.000000,2.0,56.697334,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,1.0,35.033288,0.0,0.000000,1.0,17.791063,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.000000,1.0,32.601434,0.0,0.000000,1.0,25.077050,1.0,35.120948,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
6,1.0,11.824647,1.0,25.689052,1.0,25.082700,1.0,30.703425,0.0,0.000000,...,0.0,0.0,1.0,20.666313,0.0,0.000000,0.0,0.000000,0.0,0.0
7,0.0,0.000000,1.0,31.865496,2.0,43.124539,4.0,59.959848,1.0,25.464566,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
8,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.0,32.530532,...,0.0,0.0,3.0,125.771667,0.0,0.000000,1.0,24.828332,0.0,0.0
9,0.0,0.000000,2.0,35.742970,1.0,19.069289,2.0,29.166364,0.0,0.000000,...,0.0,0.0,9.0,273.175458,0.0,0.000000,0.0,0.000000,0.0,0.0


In [37]:
volume = np.zeros((8760, 1675))
for i in tqdm(range(8760)):
    for j in range(1675):
        volume[i, j] = tf.loc[i, j*2]

  0%|          | 0/8760 [00:00<?, ?it/s]/home/skang/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
100%|██████████| 8760/8760 [05:02<00:00, 29.00it/s]


In [38]:
vol = pd.DataFrame(volume)
vol.to_csv('data/volume.csv', index=False)